# Read CSV files

In [1]:
import csv
import numpy as np
import os
import pandas as pd
import pathlib
import time

from datetime import datetime, timedelta
from pathlib import Path

In [2]:
def get_datetime(p_date, p_time):
    if (pd.notnull(p_date) & pd.notnull(p_time)):
        v_datetime = p_date + " " + p_time
        v_datetime = datetime.strptime(v_datetime, "%Y-%m-%d %H:%M:%S")
    else:
        v_datetime = np.nan
    return v_datetime

In [3]:
def get_work_duration_secs(p_ts_start, p_ts_end):
    if (pd.notnull(p_ts_start) & pd.notnull(p_ts_end)):
        v_delta = (p_ts_end - p_ts_start).total_seconds()
        if (v_delta < 0):
            v_delta = 86400 + v_delta
    else:
        v_delta = np.nan
    return v_delta

In [4]:
def get_timesheets_mast(raw_timesheets, today_str):
    today_ds = datetime.strptime(today_str, "%Y-%m-%d")
    yesterday_ds = today_ds - timedelta(days=1)
    yesterday_str = yesterday_ds.strftime("%Y-%m-%d")

    timesheets_yesterday_notnull = raw_timesheets.copy()[
        (raw_timesheets['date'] == yesterday_str) &
        (raw_timesheets['checkin'].notnull()) &
        (raw_timesheets['checkout'].notnull())
    ]
    timesheets_yesterday_notnull['checkin_ts'] = timesheets_yesterday_notnull.apply(lambda x: get_datetime(x['date'], x['checkin']), axis=1)
    timesheets_yesterday_notnull['checkout_ts'] = timesheets_yesterday_notnull.apply(lambda x: get_datetime(x['date'], x['checkout']), axis=1)
    timesheets_yesterday_notnull['work_duration_secs'] = timesheets_yesterday_notnull.apply(lambda x: get_work_duration_secs(x['checkin_ts'], x['checkout_ts']), axis=1)
   
    timesheets_yesterday_null = raw_timesheets.copy()[
        (raw_timesheets['date'] == yesterday_str) &
        ((raw_timesheets['checkin'].isnull()) |
        (raw_timesheets['checkout'].isnull()))
    ]
    timesheets_yesterday_null['work_duration_secs'] = round(timesheets_yesterday_notnull['work_duration_secs'].mean(), 2)
   
    v_cols = ['timesheet_id', 'employee_id', 'date', 'checkin', 'checkout', 'work_duration_secs']
    timesheets_yesterday = pd.concat([
        timesheets_yesterday_notnull[v_cols],
        timesheets_yesterday_null[v_cols]
    ]).reset_index(drop=True)
    timesheets_yesterday['date'] = pd.to_datetime(timesheets_yesterday['date'], format='%Y-%m-%d')
    timesheets_yesterday['year'] = timesheets_yesterday['date'].dt.year
    timesheets_yesterday['month'] = timesheets_yesterday['date'].dt.month
    timesheets_yesterday['work_hours'] = timesheets_yesterday['work_duration_secs'].apply(lambda x: round(x/3600, 2))
    
    v_cols = ['timesheet_id', 'employee_id', 'branch_id', 'monthly_salary', 'date', 'year', 'month', 'checkin', 'checkout', 'work_duration_secs', 'work_hours']
    timesheets_employees_yesterday = pd.merge(timesheets_yesterday, employees, left_on='employee_id', right_on='employe_id', how='left')
    timesheets_employees_yesterday.rename(columns={'salary': 'monthly_salary'}, inplace=True)
   
    return timesheets_employees_yesterday[v_cols]

In [5]:
def get_hourly_salary_month_to_date(fact_timesheets_mast, today_str):
    today_ds = datetime.strptime(today_str, "%Y-%m-%d")
    yesterday_ds = today_ds - timedelta(days=1)
    yesterday_str = yesterday_ds.strftime("%Y-%m-%d")
    day1st_ds = datetime(yesterday_ds.year, yesterday_ds.month, 1, 0, 0, 0)
    day1st_str = day1st_ds.strftime("%Y-%m-%d")

    stg_timesheets = fact_timesheets_mast.copy()[
        (fact_timesheets_mast['date'] >= day1st_str) &
        (fact_timesheets_mast['date'] < today_str)
    ]

    stg_employee_branch = stg_timesheets[['employee_id', 'branch_id', 'monthly_salary']].drop_duplicates()
    stg_employee_branch = stg_employee_branch.groupby('branch_id')['monthly_salary'].sum().reset_index()
       
    stg_hourly_salary = stg_timesheets.groupby(['year', 'month', 'branch_id'])['work_hours'].sum().reset_index()
    stg_hourly_salary = pd.merge(stg_hourly_salary, stg_employee_branch, on='branch_id', how='inner')
    stg_hourly_salary = stg_hourly_salary[['year', 'month', 'branch_id', 'monthly_salary', 'work_hours']]
    stg_hourly_salary['salary_per_hour'] = round(stg_hourly_salary['monthly_salary'] / stg_hourly_salary['work_hours'], 2)
    stg_hourly_salary['last_update'] = today_ds

    return stg_hourly_salary

In [6]:
def get_hourly_salary_last30d(fact_timesheets_mast, today_str):
    today_ds = datetime.strptime(today_str, "%Y-%m-%d")
    yesterday_ds = today_ds - timedelta(days=1)
    yesterday_str = yesterday_ds.strftime("%Y-%m-%d")
    prev30d_ds = today_ds - timedelta(days=30)
    prev30d_str = prev30d_ds.strftime("%Y-%m-%d")

    stg_timesheets = fact_timesheets_mast.copy()[
        (fact_timesheets_mast['date'] >= prev30d_str) &
        (fact_timesheets_mast['date'] < today_str)
    ]

    stg_employee_branch = stg_timesheets[['employee_id', 'branch_id', 'monthly_salary']].drop_duplicates()
    stg_employee_branch = stg_employee_branch.groupby(['branch_id'])['monthly_salary'].sum().reset_index()
       
    stg_hourly_salary = stg_timesheets.groupby(['branch_id'])['work_hours'].sum().reset_index()
    stg_hourly_salary.rename(columns={'work_hours': 'l30d_work_hours'}, inplace=True)
    stg_hourly_salary = pd.merge(stg_hourly_salary, stg_employee_branch, on='branch_id', how='inner')
    stg_hourly_salary['l30d_salary_per_hour'] = round(stg_hourly_salary['monthly_salary'] / stg_hourly_salary['l30d_work_hours'], 2)
    stg_hourly_salary['date_start'] = prev30d_ds
    stg_hourly_salary['date_end'] = yesterday_ds
    stg_hourly_salary['last_update'] = today_ds

    return stg_hourly_salary[['date_start', 'date_end', 'branch_id', 'monthly_salary', 'l30d_work_hours', 'l30d_salary_per_hour', 'last_update']]

In [7]:
src_employees = f"{os.path.dirname(os.getcwd())}\src\\employees.csv"
src_timesheets = f"{os.path.dirname(os.getcwd())}\src\\timesheets.csv"

(src_employees, src_timesheets)

('C:\\Users\\ZALDI\\Documents\\Pekerjaan Profesional\\Coding Test\\Mekari\\src\\employees.csv',
 'C:\\Users\\ZALDI\\Documents\\Pekerjaan Profesional\\Coding Test\\Mekari\\src\\timesheets.csv')

In [8]:
employees = pd.read_csv(src_employees, sep=',')
employees

,employe_id,branch_id,salary,join_date,resign_date
0,1,3,7500000,2018-08-23,NaN
1,7,1,7500000,2017-04-28,NaN
2,8,1,13000000,2017-04-28,NaN
3,9,1,13500000,2017-12-22,2020-10-14
4,10,1,9000000,2018-03-01,2020-12-20
...,...,...,...,...,...
172,311335,1,13500000,2020-11-16,NaN
173,325787,1,12500000,2020-12-03,NaN
174,329501,1,13000000,2020-12-07,NaN
175,331374,2629,6000000,2020-12-07,NaN


In [9]:
timesheets = pd.read_csv(src_timesheets, sep=',')
timesheets

,timesheet_id,employee_id,date,checkin,checkout
0,23907432,66,2019-08-21,08:13:31,17:05:02
1,23907433,22,2019-08-21,08:56:34,18:00:48
2,23907434,21,2019-08-21,09:45:08,18:24:06
3,23907435,63,2019-08-21,09:55:47,NaN
4,23907437,60,2019-08-21,09:56:05,17:31:08
...,...,...,...,...,...
39709,70649593,59,2020-12-23,08:05:29,17:05:18
39710,70649597,109100,2020-12-23,08:05:30,17:05:30
39711,70650186,9908,2020-12-23,08:06:52,NaN
39712,70650458,70,2020-12-23,08:07:33,17:13:27


In [10]:
fact_timesheets_cols = {
    'timesheet_id': pd.Series(dtype='int64'),
    'employee_id': pd.Series(dtype='int64'),
    'branch_id': pd.Series(dtype='int64'),
    'monthly_salary': pd.Series(dtype='int64'),
    'date': pd.Series(dtype='str'),
    'year': pd.Series(dtype='int64'),
    'month': pd.Series(dtype='int64'),
    'checkin': pd.Series(dtype='str'),
    'checkout': pd.Series(dtype='str'),
    'work_duration_secs': pd.Series(dtype='float64'),
    'work_hours': pd.Series(dtype='float64')
}
fact_timesheets = pd.DataFrame(fact_timesheets_cols)
fact_timesheets_path = f"{os.path.dirname(os.getcwd())}\\out\\FACT_TIMESHEETS\\"
if (not os.path.exists(fact_timesheets_path)):
    os.makedirs(fact_timesheets_path)

In [11]:
fact_hourly_salary_month_to_date_cols = {
    'year': pd.Series(dtype='int64'),
    'month': pd.Series(dtype='int64'),
    'branch_id': pd.Series(dtype='int64'),
    'monthly_salary': pd.Series(dtype='int64'),
    'work_hours': pd.Series(dtype='float64'),
    'salary_per_hour': pd.Series(dtype='float64'),
    'last_update': pd.Series(dtype='str')
}
fact_hourly_salary_month_to_date = pd.DataFrame(fact_hourly_salary_month_to_date_cols)
fact_hourly_salary_month_to_date_path = f"{os.path.dirname(os.getcwd())}\\out\\FACT_HOURLY_SALARY_MONTH_TO_DATE\\"
if (not os.path.exists(fact_hourly_salary_month_to_date_path)):
    os.makedirs(fact_hourly_salary_month_to_date_path)

In [12]:
fact_hourly_salary_last30d_cols = {
    'date_start': pd.Series(dtype='str'),
    'date_end': pd.Series(dtype='str'),
    'branch_id': pd.Series(dtype='int64'),
    'monthly_salary': pd.Series(dtype='int64'),
    'l30d_work_hours': pd.Series(dtype='float64'),
    'l30d_salary_per_hour': pd.Series(dtype='float64'),
    'last_update': pd.Series(dtype='str')
}
fact_hourly_salary_last30d = pd.DataFrame(fact_hourly_salary_last30d_cols)
fact_hourly_salary_last30d_path = f"{os.path.dirname(os.getcwd())}\\out\\FACT_HOURLY_SALARY_LAST30D\\"
if (not os.path.exists(fact_hourly_salary_last30d_path)):
    os.makedirs(fact_hourly_salary_last30d_path)

In [13]:
list_ds = []
ds_start = "2019-08-01" # "YYYY-MM-DD" FORMAT. INCLUSIVE.
ds_end = "2020-01-01" # "YYYY-MM-DD" FORMAT. INCLUSIVE.

try:
    ds_start = datetime.strptime(ds_start, '%Y-%m-%d')
    ds_end = datetime.strptime(ds_end, '%Y-%m-%d')
except Exception as e:
    print(e)
else:
    while (ds_start <= ds_end):
        list_ds.append(ds_start.strftime('%Y-%m-%d'))
        ds_start += timedelta(days=1)

list_ds

['2019-08-01',
 '2019-08-02',
 '2019-08-03',
 '2019-08-04',
 '2019-08-05',
 '2019-08-06',
 '2019-08-07',
 '2019-08-08',
 '2019-08-09',
 '2019-08-10',
 '2019-08-11',
 '2019-08-12',
 '2019-08-13',
 '2019-08-14',
 '2019-08-15',
 '2019-08-16',
 '2019-08-17',
 '2019-08-18',
 '2019-08-19',
 '2019-08-20',
 '2019-08-21',
 '2019-08-22',
 '2019-08-23',
 '2019-08-24',
 '2019-08-25',
 '2019-08-26',
 '2019-08-27',
 '2019-08-28',
 '2019-08-29',
 '2019-08-30',
 '2019-08-31',
 '2019-09-01',
 '2019-09-02',
 '2019-09-03',
 '2019-09-04',
 '2019-09-05',
 '2019-09-06',
 '2019-09-07',
 '2019-09-08',
 '2019-09-09',
 '2019-09-10',
 '2019-09-11',
 '2019-09-12',
 '2019-09-13',
 '2019-09-14',
 '2019-09-15',
 '2019-09-16',
 '2019-09-17',
 '2019-09-18',
 '2019-09-19',
 '2019-09-20',
 '2019-09-21',
 '2019-09-22',
 '2019-09-23',
 '2019-09-24',
 '2019-09-25',
 '2019-09-26',
 '2019-09-27',
 '2019-09-28',
 '2019-09-29',
 '2019-09-30',
 '2019-10-01',
 '2019-10-02',
 '2019-10-03',
 '2019-10-04',
 '2019-10-05',
 '2019-10-

In [14]:
ts_log_begin = time.time()
for dd in list_ds:
    ds_yyyymmdd = dd.replace("-", "")
    
    out = get_timesheets_mast(timesheets, dd)
    # out.to_csv(f"{fact_timesheets_path}\\FACT_TIMESHEETS_{ds_yyyymmdd}.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)
    fact_timesheets = pd.concat([fact_timesheets, out], ignore_index=True)
    
    out = get_hourly_salary_month_to_date(fact_timesheets, dd)
    # out.to_csv(f"{fact_hourly_salary_month_to_date_path}\\FACT_HOURLY_SALARY_MONTH_TO_DATE_{ds_yyyymmdd}.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)
    fact_hourly_salary_month_to_date = pd.concat([fact_hourly_salary_month_to_date, out], ignore_index=True)
    
    out = get_hourly_salary_last30d(fact_timesheets, dd)
    # out.to_csv(f"{fact_hourly_salary_last30d_path}\\FACT_HOURLY_SALARY_LAST30D_{ds_yyyymmdd}.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)
    fact_hourly_salary_last30d = pd.concat([fact_hourly_salary_last30d, out], ignore_index=True)
ts_log_end = time.time()

print(f"Simulation finished in {round(ts_log_end-ts_log_begin, 3)} seconds.")

Simulation finished in 5.669 seconds.


In [15]:
fact_timesheets

,timesheet_id,employee_id,branch_id,monthly_salary,date,year,month,checkin,checkout,work_duration_secs,work_hours
0,23907432,66,2,10000000,2019-08-21,2019,8,08:13:31,17:05:02,31891.00,8.86
1,23907433,22,1,8000000,2019-08-21,2019,8,08:56:34,18:00:48,32654.00,9.07
2,23907434,21,1,13500000,2019-08-21,2019,8,09:45:08,18:24:06,31138.00,8.65
3,23907437,60,2,12500000,2019-08-21,2019,8,09:56:05,17:31:08,27303.00,7.58
4,23907435,63,2,13500000,2019-08-21,2019,8,09:55:47,NaN,30746.50,8.54
...,...,...,...,...,...,...,...,...,...,...,...
8261,29804437,34514,2633,14500000,2019-12-31,2019,12,07:29:46,15:17:10,28044.00,7.79
8262,30544720,53191,2635,6000000,2019-12-31,2019,12,07:01:38,14:16:33,26095.00,7.25
8263,30560587,49616,3092,10000000,2019-12-31,2019,12,15:11:00,06:00:00,53340.00,14.82
8264,29488728,41,2623,13500000,2019-12-31,2019,12,NaN,06:01:22,28068.62,7.80


In [16]:
fact_hourly_salary_month_to_date

,year,month,branch_id,monthly_salary,work_hours,salary_per_hour,last_update
0,2019,8,1,21500000,17.72,1213318.28,2019-08-22
1,2019,8,2,36000000,24.98,1441152.92,2019-08-22
2,2019,8,1,21500000,26.82,801640.57,2019-08-23
3,2019,8,2,36000000,42.83,840532.34,2019-08-23
4,2019,8,1,21500000,35.88,599219.62,2019-08-24
...,...,...,...,...,...,...,...
1562,2019,12,2633,62500000,978.65,63863.49,2020-01-01
1563,2019,12,2634,10000000,181.69,55038.80,2020-01-01
1564,2019,12,2635,35500000,577.20,61503.81,2020-01-01
1565,2019,12,3092,93000000,2204.51,42186.25,2020-01-01


In [17]:
fact_hourly_salary_last30d

,date_start,date_end,branch_id,monthly_salary,l30d_work_hours,l30d_salary_per_hour,last_update
0,2019-07-23,2019-08-21,1,21500000,17.72,1213318.28,2019-08-22
1,2019-07-23,2019-08-21,2,36000000,24.98,1441152.92,2019-08-22
2,2019-07-24,2019-08-22,1,21500000,26.82,801640.57,2019-08-23
3,2019-07-24,2019-08-22,2,36000000,42.83,840532.34,2019-08-23
4,2019-07-25,2019-08-23,1,21500000,35.88,599219.62,2019-08-24
...,...,...,...,...,...,...,...
1570,2019-12-02,2019-12-31,2633,62500000,940.13,66480.17,2020-01-01
1571,2019-12-02,2019-12-31,2634,10000000,174.74,57227.88,2020-01-01
1572,2019-12-02,2019-12-31,2635,35500000,569.37,62349.61,2020-01-01
1573,2019-12-02,2019-12-31,3092,93000000,2149.83,43259.23,2020-01-01
